# Pollywog Workflow Tutorial

This notebook demonstrates pollywog workflows for mining geometallurgy: preprocessing drillhole data, postprocessing block model grades, modeling recovery with regression trees, using helper functions, and visualizing calculation sets.

In [ ]:
import pollywog as pw

## 1. Preprocessing Drillhole Data

Import pollywog and preprocess raw assay data for Au, Ag, and Cu using CalcSet and Number. Export the preprocessing CalcSet for Leapfrog.

In [ ]:
from pollywog.core import CalcSet, Number

variables = ["Au", "Ag", "Cu"]
preprocess = CalcSet(
    [
        *[Number(f"{v}_clean", f"clamp([{v}], 0)") for v in variables],
        *[Number(f"{v}_log", f"log([{v}_clean] + 1e-6)") for v in variables],
    ]
)

# Export for use in Leapfrog (drillholes)
preprocess.to_lfcalc("drillhole_preprocessing.lfcalc")

## 2. Postprocessing Block Model Grades

Calculate final grades as weighted averages of domain grades using manual normalization and the WeightedAverage helper. Export both CalcSets for Leapfrog.

In [ ]:
from pollywog.helpers import WeightedAverage

variables = ["Au", "Ag", "Cu"]
domains = ["high", "medium", "low"]
# Manual normalization
postprocess_manual = CalcSet(
    [
        *[
            Number(
                f"{v}_final",
                f"(({' + '.join([f'[prop_{d}] * [{v}_{d}]' for d in domains])}) / ({' + '.join([f'[prop_{d}]' for d in domains])}))",
            )
            for v in variables
        ],
    ]
)
# Using WeightedAverage helper
postprocess_helper = CalcSet(
    [
        *[
            WeightedAverage(
                variables=[f"{v}_{d}" for d in domains],
                weights=[f"prop_{d}" for d in domains],
                name=f"{v}_final_weighted",
            )
            for v in variables
        ],
    ]
)

# Export for use in Leapfrog (block model)
postprocess_manual.to_lfcalc("blockmodel_postprocessing_manual.lfcalc")
postprocess_helper.to_lfcalc("blockmodel_postprocessing_weighted.lfcalc")

## 3. Geometallurgical Recovery with Regression Trees

Create example data, fit a regression tree for Au recovery, convert the tree to a CalcSet, and add it to the block model CalcSets. Export updated CalcSets.

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from pollywog.conversion.sklearn import convert_tree
from pollywog.core import CalcSet

# Example data: columns are Au_final, Ag_final, Cu_final
X = np.array(
    [
        [1.2, 5.0, 0.3],
        [0.8, 2.5, 0.1],
        [2.0, 1.0, 0.5],
        [0.5, 3.2, 0.2],
        [1.5, 4.1, 0.4],
        [0.3, 0.8, 0.05],
        [2.5, 2.0, 0.7],
        [1.8, 3.5, 0.6],
        [0.9, 1.2, 0.2],
        [1.0, 2.8, 0.3],
    ]
)
# Example recoveries for Au (could be based on lab tests)
y_au = np.array([0.85, 0.78, 0.92, 0.75, 0.88, 0.65, 0.95, 0.90, 0.80, 0.83])

# Fit regression tree for Au recovery
tree_au = DecisionTreeRegressor(max_depth=3)
tree_au.fit(X, y_au)

# Convert tree to CalcSet
tree_calcset = CalcSet(
    convert_tree(
        tree_au,
        ["Au_final", "Ag_final", "Cu_final"],
        "Au_recovery",
    )
)

# Add recovery calculation to block model calcsets
postprocess_manual.items += tree_calcset.items
postprocess_helper.items += tree_calcset.items

# Export updated block model calcsets
postprocess_manual.to_lfcalc("blockmodel_postprocessing_with_recovery_manual.lfcalc")
postprocess_helper.to_lfcalc("blockmodel_postprocessing_with_recovery_weighted.lfcalc")

## 4. Helper Functions for Block Model Calculations

Demonstrate Sum, Product, Normalize, Scale, IfElse, and CategoryFromThresholds helpers. Add them to a CalcSet and export.

In [ ]:
from pollywog.core import If, Number
from pollywog.helpers import (
    Sum,
    Product,
    Normalize,
    Scale,
    CategoryFromThresholds,
)

# Sum: Add several variables together
sum_example = Sum(["Au_final", "Ag_final", "Cu_final"], name="Total_Metals")

# Product: Multiply variables (e.g., grade * recovery)
product_example = Product(["Au_final", "Au_recovery"], name="Au_payable")

# Normalize: Min-max normalization of a variable to [0, 1]
normalize_example = Normalize("Au_final", min_value=0.0, max_value=5.0, name="Au_normalized")

# Scale: Apply a scaling factor to a variable
scale_example = Scale("Au_final", 0.95, name="Au_final_scaled")

# If: Conditional logic (e.g., flag blocks above cutoff)
from pollywog.core import Category
if_example = Category(
    "OreFlag",
    If(("[Au_final] > 0.5", "'Ore'"), "'Waste'")
)

# CategoryFromThresholds: Categorize based on thresholds
cat_example = CategoryFromThresholds(
    variable="Au_final",
    thresholds=[0.3, 1.0],
    categories=["Low", "Medium", "High"],
    name="AuCategory",
)

# Add these to a CalcSet and export
helpers_calcset = CalcSet(
    [
        sum_example,
        product_example,
        normalize_example,
        scale_example,
        if_example,
        cat_example,
    ]
)
helpers_calcset.to_lfcalc("blockmodel_helpers_examples.lfcalc")

## 5. Visualization of CalcSets

Visualize any CalcSet in Jupyter using pollywog.display functions.

In [ ]:
from pollywog.display import display_calcset, set_theme

set_theme("light")
display_calcset(preprocess)
display_calcset(postprocess_manual)
display_calcset(postprocess_helper)
display_calcset(helpers_calcset)